# Julia avanzado

## Metaprogramming

Julia, igual que Lisp, representa al código como una estructura de datos en el propio lenguaje. La posibilidad de escribir código que genere y modifique código es lo que se entiende por "Metaprogramming".

Ilustraremos algunos conceptos siguiendo el [manual](http://docs.julialang.org/en/release-0.4/manual/metaprogramming/). Cualquier línea de código, inicialmente es una cadena:

In [ ]:
one_plus_one = "1+1"

El siguiente paso es convertir esta cadena en una *expresión*:

In [ ]:
ex1 = parse(one_plus_one)

In [ ]:
typeof(ex1)

In [ ]:
?Expr

Claramente, un objeto tipo `Expr` tiene tres partes:

- Un `Symbol`, que se guarda en `head`:

In [ ]:
ex1.head

- Un vector (tipo `Any`) que contiene a los argumentos:

In [ ]:
ex1.args

- el tipo resultante de la expresión, que puede ser anotado por el usuario

In [ ]:
ex1.typ

Las expresiones también pueden ser escritas directamente:

In [ ]:
ex2 = Expr(:call, :+, 1, 1)

El punto importante es que el código en Julia está representado internamente por expresiones escritas en Julia.

La función `dump()` da información sobre la expresión:

In [ ]:
dump(ex2)

Expresiones más complicadas pueden ser construidas:

In [ ]:
ex3 = parse("(4 + 4) / 2")

In [ ]:
Meta.show_sexpr(ex3)

Uno de los usos de `:` es crear símbolos; uno también puede usar `symbol()`:

In [ ]:
:foo == symbol("foo")

In [ ]:
symbol(:var,'_',"sym")

Otro uso de `:` es crear expresiones sin usar `Expr`; a esto se le llama *quoting*.

In [ ]:
ex = :(a+b*c+1)

In [ ]:
dump(ex)

Lo interesante, es que uno puede cambiar *programáticamente* el contenido de una expresión. Por ejemplo, en lugar de $ex=a + b*c + 1$ quiero tener $ex=a + b*c + 2.1$:

In [ ]:
ex.args[4] = 2.1

In [ ]:
dump(ex)

Otra alternativa para definir expresiones complejas es usando el comando `quote`

In [ ]:
# Lo siguiente es equivalente a:
# ex = :(x=1; y=2; x+y)
ex = quote
    x = 1
    y = 2
    x + y
end

In [ ]:
dump(ex)


Para evaluar una expresión, usamos `eval()`. Respecto a esto, vale la pena notar que, mientras no se ha evaluado una expresión las variables internas no tienen ningún valor; sin embargo, una vez evaluada, los valores de las variables internas quedan definidas. Esto puede causar problemas, llamados *side effects*.

In [ ]:
x

In [ ]:
y = 3

In [ ]:
eval(ex)

In [ ]:
x

In [ ]:
y # y queda sobreescrita al evaluar la expresión !!

Uno incluso puede definir el valor de ciertas variables y usarlo como tal, o sustituirlo directamente:

In [ ]:
z = 4

In [ ]:
:(2*x+z)

In [ ]:
eval(ans)

In [ ]:
:(2*x+$z)

In [ ]:
:(2*x+$zz)

El punto es que uno puede definir funciones, que regresen expresiones, que pueden ser evaluadas

Un ejemplo más interesante es implementar los polinomios de Wilkinson, que se definen como:

\begin{equation}
    W_n(x) = \prod_{i=1}^n (x-i) = (x-1) (x-2) \cdots (x-n).
\end{equation}

Ciertamente este ejemplo se puede implementar a través de una recursión, pero es un buen ejemplo.

In [ ]:
nombre(n::Int) = symbol( string("W_", n) )

In [ ]:
nombre(3)

In [ ]:
function wilkinson(n::Int)
    # Checo que `n` sea >= 1
    @assert n >= 1
    
    ex = :(x-1)
    for i = 2:n
        ex = :( ($ex) * ( x-$i) )
    end
    ex_ret = :( $(nombre(n))(x) = $ex )
    ex_ret
end

In [ ]:
wilkinson(0)

In [ ]:
wilkinson(3)

In [ ]:
eval(ans)

In [ ]:
W_3(2)

In [ ]:
eval(wilkinson(4))

In [ ]:
W_4(2.5)

**Ejercicio 7:** Implementen el factorial con metaprogramming.

## Macros

En varias ocasiones hemos usado ya algunas instrucciones que incluyen `@` antes, por ejemplo, `@time`. Estos son los macros: Los macros son funciones cuyas entradas son *expresiones*, que son manipuladas y al final se evalúan.

In [ ]:
macro simple_example(expr)
    @show expr   # this is another macro !
    return 0     # for simplicity
end

In [ ]:
@simple_example(x+y)

In [ ]:
@simple_example a = x + y

Cambiemos un poco el macro `@simple_example`

In [ ]:
macro simple_example(expr)
    @show expr   # this is another macro !
    expr         # returns the input expression
end

In [ ]:
@simple_example x + y

In [ ]:
x = 3; y = 7;
@simple_example x + y

La gran sutileza de los macros es que, a diferencia de las funciones genéricas, permiten introducir y modificar código *antes* de que sea ejecutado, dado que los macros son ejecutados cuando el código se pasa (*parse time*).

El siguiente ejemplo está tomado del manual.

In [ ]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)

    return :(println("I execute at runtime. The argument is: ", $arg))
end

In [ ]:
ex = macroexpand( :(@twostep :(1, 2, 3)) );

In [ ]:
macroexpand( (:@simple_example x+y) )

In [ ]:
typeof(ex)

In [ ]:
ex

In [ ]:
eval(ex)

In [ ]:
?@time

## Generación de código

Arriba, con la definición `wilkinson()` de hecho definimos una función que genera código. Si bien ése no es el mejor ejemplo para esto, el punto es que a veces, excepto por pequeños cambios (por ejemplo, en los operadores), el código esencialmente se repite.

Un ejemplo de lo anterior es el siguiente:

In [ ]:
type Vector2D{T<:Real}
    x :: T
    y :: T
end

In [ ]:
Vector2D(a,b) = Vector2D(promote(a,b)...)

In [ ]:
import Base: +, -

for fn = (:+, :-)
    println(fn)
    ex = quote
        function ($fn)(a::Vector2D, b::Vector2D)
            xx = ($fn)(a.x, b.x)
            yy = ($fn)(a.y, b.y)
            return Vector2D(xx, yy)
        end
    end
    println(ex)
    @eval $ex
end

In [ ]:
Vector2D(1, 2.2) - Vector2D(1,1)